In [ ]:
import gym
from gym import logger as gymlogger
from gym.wrappers import RecordVideo
gymlogger.set_level(40) #error only
import tensorflow as tf
import numpy as np
import random
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import math
import glob
import io
import base64
from IPython.display import HTML
from IPython import display as ipythondisplay
import pygame
from typing import List

In [ ]:
env = gym.make("CartPole-v1"
            #    , render_mode="human"
               )

In [ ]:
print(f"this is observation space: {type(env.observation_space)}")
print(f"this is action space: {env.action_space}")

# 0: cart position 1: cart velocity 2: pole angle 3: pole angular velocity
# observation high represents the highest value that the observation can take
print(f"this is observation space high: {env.observation_space.high}")
upper_bound = env.observation_space.high.tolist()

# observation lowest represents the lowest value that the observation can take
print(f"this is observation space low: {env.observation_space.low}")
lower_bound = env.observation_space.low.tolist()

In [ ]:
def random_seed() -> int:
    return random.randint(0, 10000)

seed = random_seed()
print(f"this is the seed: {seed}")
env.action_space.seed(seed)
state = env.reset()[0]
print(state)
print(state[0])
print(env.step(0)[0])

# print(type(env))
# print(env.action_space.n)
# print(env.observation_space.shape)
# print(env.observation_space.shape[0])

### Task 1: need to choose corresponding action according to state of the cartpole

##### Task 1.1: getting discrete state

State space in cart pole is infinite, which means it is continuous

hence we need to discretise the state space if not there will be infinite rows in the state space table

We will need to discretise each state variable into n segments

potentially have many different states being mapped into 1 state, but we can make the states finer inorder to reduce the amount of different states being sent into this table

In [ ]:
# change the values inside the tuple for the number of segments in each bucket (the more the better)
n_buckets = (10,10,10,10)

In [ ]:
def discretiser(
        cart_position: float, 
        cart_velocity: float, 
        angle: float, 
        angular_velocity: float, 
        n_buckets: tuple = (10,10,10,10),
        lower_bounds: list = env.observation_space.low.tolist(),
        upper_bounds: list = env.observation_space.high.tolist(),
    ) -> tuple:
    """
    Discretises the continuous state space into discrete states for CartPole-v1.
    
    Parameters:
    - cart_position (float): The cart's position.
    - cart_velocity (float): The cart's velocity.
    - angle (float): The pole's angle.
    - angular_velocity (float): The pole's angular velocity.
    - n_buckets (tuple): Number of bins for each state variable.
    - lower_bounds (list): Lower bounds for each state variable. [OPTIONAL]
    - upper_bounds (list): Upper bounds for each state variable. [OPTIONAL]

    Returns:
    - tuple: Discretized indices for (cart_position, cart_velocity, angle, angular_velocity).
    """

    # specify the width of each bucket for each state variable
    bucket_width = [(upper_bounds[i] - lower_bounds[i]) / (n_buckets[i] - 1) for i in range(len(n_buckets))]
    
    # discretise the state space
    cart_pos_index = int(min(max((cart_position - lower_bounds[0]) / bucket_width[0], 0), n_buckets[0] - 1))
    cart_vel_index = int(min(max((cart_velocity - lower_bounds[1]) / bucket_width[1], 0), n_buckets[1] - 1))
    angle_index = int(min(max((angle - lower_bounds[2]) / bucket_width[2], 0), n_buckets[2] - 1))
    angular_vel_index = int(min(max((angular_velocity - lower_bounds[3]) / bucket_width[3], 0), n_buckets[3] - 1))


    return (cart_pos_index, cart_vel_index, angle_index, angular_vel_index)


##### Task 1.2: Q-table definition

In [ ]:
Q_table = np.random.uniform(low=0, high=1, size=(n_buckets[0],n_buckets[1],n_buckets[2],n_buckets[3],env.action_space.n))

##### Task 1.3: selection of action

In [ ]:
global epsilon

def selectAction(state, index: int, epsilon: float) -> int:

    # letting the first 300 episodes to do completely random actions to explore the environment
    if index < 300:
        return np.random.choice(env.action_space.n)
    
    # this number will be used for epsilon greedy strategy
    random_number = np.random.random()

    # after 7000 episodes, decrease the epsilon parameter
    if index > 7000:
        epsilon = 0.999 * epsilon

    # if the condition is met, then choose a random action
    if random_number < epsilon:
        return np.random.choice(env.action_space.n)

    # if the condition is not met, then choose the action with the highest Q value
    else:
        return np.random.choice(np.where(Q_table[state] == Q_table[state].max())[0])

In [ ]:
def simulate_episodes(number_of_episodes: int):
    for index_episodes in range(number_of_episodes):

        reward_episodes = []
        state_s = env.reset()[0]

        is_terminated = False
        while not is_terminated:

            state_s_index = discretiser(state_s[0], state_s[1], state_s[2], state_s[3])

            action = selectAction(state_s_index, index_episodes, 0.9)
            next_state, reward, is_terminated, _ = env.step(action)
            reward_episodes.append(reward)

            next_state_index = discretiser(next_state[0], next_state[1], next_state[2], next_state[3])

            q_max_prime = np.max(Q_table[next_state_index])

            if not is_terminated:
                Q_table[state_s_index][action] += 0.1 * (reward + 0.99 * q_max_prime - Q_table[state_s_index][action])

### Task 2: run the RL agent 100 times, reset state at the start of each iteration 

### Task 3: Render one episode played by the developed RL agent on Jupyter.